In [1]:
import gurobipy as gp
from gurobipy import GRB
from IPython.display import display, Math, Latex

import import_ipynb
import data_utils as data
tt = gp.Model('IIITB Exams Timetable')

/home/owais/miniconda3/envs/timetabling/lib/python3.10/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2615624
Academic license 2615624 - for non-commercial use only - registered to mo___@iiitb.ac.in


## Make one binary variable for each room-course-timeslot-day combination
$$
E_{crtd} =
\begin{cases} 
1 & \text{if a course $c$ exam is scheduled in room $r$ at time $t$ on day $d$,} \\
0 & \text{Otherwise.}
\end{cases}
$$

In [2]:
#Dhruv's part:
exam_spans = {c: data.begin_end_pairs(data.get_exam_duration(c)) for c in range(data.n_courses)}

E = tt.addVars(
    [
        (c ,b ,e, d)
        for c in range(data.n_courses)
        for b, e in exam_spans[c]
        for d in range(data.n_days)
    ],
    vtype=GRB.BINARY,
    name="E",
)

C = tt.addVars(
    [
        (c,r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.INTEGER,
    lb=0,
    name="C",
)


KeyError: 'ExamDuration'

## Hard Constraints

In [26]:
# Exactly one slot for each course
# sum of enrolments in courses assigned to a room <= room capacity
# no clash for any student
# all slots for a course on any day are contiguous
# exam lengths for each course


# ---Modified Constraints--- (Sarvesh)
# Exactly one continuous slot for each course satisying the length of the course
# Exactly one room for each course
# Exactly one day for each course

## Hard Constraint 1: Exactly one continuous slot for each course satisying the length of the course (Sarvesh)

$$
\sum_{r=1}^{\text{data.n\_rooms}}\sum_{d=1}^{\text{data.n\_days}} \sum_{t=1}^{\text{data.n\_times} - \text{exam\_slots}(c) + 1} \sum_{t' = t}^{t + \text{exam\_slots}(c) - 1} 
\text{feasibility\_exam\_slot}(\text{times}[t], \text{times}[t + \text{exam\_slots}(c) - 1], c) \cdot E_{crt'd} = \text{exam\_slots}(c), 
$$


$$
\sum_{r=1}^{\text{data.n\_rooms}}\sum_{d=1}^{\text{data.n\_days}} \sum_{t=1}^{\text{data.n\_times}} E_{crtd} = \text{exam\_slots}(c), 
$$

$$
\forall c \in \{1, \dots, \text{data.n\_courses}\}.
$$




In [27]:
# # Hard Constraint 1: Exactly one continuous slot for each course satisying the length of the course (Sarvesh)


# ------------------------ Exam duration of a course doesn't exceed time slot allocated for the course------------------------
def exam_length(course:int):
    return data.get_exam_duration(course)
    #returns the exam length of the course given the course id (in the 'course' dataframe)

def exam_slots(course:int):
    # return (data.get_exam_duration(course)//15)
    return 6
    #returns the number of slots required for the course given the course id (in the 'course' dataframe)


def signum(x):
    if x>0:
        return 1
    return 0

#Dhruv's Part:

tt.addConstrs(
    (E.sum(c, '*', '*', '*') == 1 for c in range(data.n_courses)),
    name="one_exam_per_course",
)

{}

## Hard Constraint 2: Sum of enrolments in courses assigned to a room <= room capacity (Sarvesh)

$$
\sum_{c=1}^{\text{data.n\_courses}} \text{enrollments\_in\_course}(c) \cdot E_{crtd} \leq \text{room\_capacity}(r)
$$

$$
\forall r \in \{1, \dots, \text{data.n\_rooms}\}, \forall d \in \{1, \dots, \text{data.n\_days}\}, \forall t \in \{1, \dots, \text{data.n\_times}\}.
$$

In [28]:
# Hard Constraint 2: Sum of enrolments in courses assigned to a room <= room capacity (Sarvesh)


def room_capacity(room:int):
    return data.get_room_capacity(room)
    #returns the capacity of the room given the room id (in the 'room' dataframe)
    

# Each exam is scheduled can be scheduled in atmost 3 rooms but at least 1 room
tt.addConstrs(
    (C.sum(signum(c, '*')) <= 3 for c in range(data.n_courses)),
    name="atmost_3_rooms_per_course",
)
tt.addConstrs(
    (C.sum(signum(c, '*')) >= 1 for c in range(data.n_courses)),
    name="atleast_1_room_per_course",
)

# Capacity constraint - Multiple courses in a single room

tt.addConstrs(
    (gp.quicksum(C[c,r]*E[c,b,e,d]
     for c in range(data.n_courses)
     for r in range(data.n_rooms) <= room_capacity(r)
     for b,e in exam_spans[c] 
     for d in range(data.n_days)
     )
    ),
    name="room_capacity_constraint",

)

{}

## Hard Constraint 3: No Clash for any student (Sarvesh)

$$
\sum_{\substack{c=1 \\ \text{if student $s$ is taking course $c$}}}^{\text{data.n\_courses}} 
\sum_{r=1}^{\text{data.n\_rooms}} E_{crtd} \leq 1,
$$

$$
\forall s \in \{1, \dots, \text{data.n\_students}\}, \forall d \in \{1, \dots, \text{data.n\_days}\}, \forall t \in \{1, \dots, \text{data.n\_times}\}.
$$

In [29]:
# Hard Constraint 3: No clash for any student (Sarvesh)

def check_student_taking_course(student:int,course:int):
    #returns True if the student is taking the course given the student id and course id (in the 'student' and 'course' dataframe)
    return data.check_student_taking_course(student,course)



def covering_sessions(s:int, t:int):
    # Return the list of begin-end pairs for session 's' overlapping with time 't'
    return [(b,e) for b,e in exam_spans[s] if not overlap(b,e,t)]

tt.addConstrs((gp.quicksum(signum(C[c,r])*E[c,b,e,d]
                          for c in range(data.n_courses)
                          for r in range(data.n_rooms)
                          for b,e in covering_sessions(s, t)
                          if check_student_taking_course(s,c)
                          ) <= 1
              for d in range(data.n_days)
              for s in range(data.n_students)
              for t in range(data.n_times)),
              name='room_double_booking'
             )



{}

In [ ]:
# Constraint 1: Each exam is scheduled exactly once
tt.addConstrs(
    (E.sum(c, '*', '*', '*') == 1 for c in range(data.n_courses)),
    name="one_exam_per_course",
)

# Constraint 2: Room capacity constraint C[c, r]*E[c, b, e, d] <= room_capacity[r] for all rooms

tt.addConstrs(
    # (
    #     C[c, r] * E[c, b, e, d] <= data.room_capacities[r]
    #     for c in range(data.n_courses)
    #     for r in range(data.n_rooms)
    #     for b, e in exam_spans[c]
    #     for d in range(data.n_days)
    # ),
    (
        C['*', r] * E.sum('*', '*', '*', '*') <= data.room_capacities[r] 
        for r in range(data.n_rooms)
    ),
    name="room_capacity_constraint",
)

# Constraint 3: All students for a course should write the exam 
tt.addConstrs(
    # (
    #     C[c, r] * E[c, b, e, d] <= data.enrolments[c]
    #     for c in range(data.n_courses)
    #     for r in range(data.n_rooms)
    #     for b, e in exam_spans[c]
    #     for d in range(data.n_days)
    # ),
    (
        C[c, '*'] * E.sum(c, '*', '*', '*') <= data.enrolments[c]
        for c in range(data.n_courses)
    ),
    name="course_enrollment_constraint",
)

# Constraint 4: Student clash constraint

# for each student, for each day, for each time slot, at most one exam
# sigma Ecbed <= 1 for courses enrolled by student s

tt.addConstrs(
    (
        gp.quicksum(E[c, '*', '*', '*'] for c in data.student_courses[s]) <= 1
        for s in range(data.n_students)
    ),
    name="student_clash_constraint",
)

# No course can have more than 3 rooms alloted
tt.addConstrs(
    (
        gp.quicksum(signum(C[c, r]) for r in range(data.n_rooms)) <= 3
        for c in range(data.n_courses)
    ),
    name="course_room_constraint",
)







## Soft Constraints

In [30]:
# preferred time slots for exams

## Objective Function

In [31]:
# Maximize Gaps between exams for a student / batch
tt.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.1 LTS")

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 0 rows, 0 columns and 0 nonzeros
Model fingerprint: 0xf9715da1
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.03 seconds (0.00 work units)
Optimal objective  0.000000000e+00
